In [6]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
import polars as pl
from datetime import datetime
from IPython.display import display

registros = pl.read_csv("../DailyWeatherData.csv").to_pandas()

registros["Fecha"] = pd.to_datetime(registros["Fecha"], format="%Y-%m-%d", errors="coerce")
registros = registros.dropna(subset=["Fecha"])

sensores = pd.read_csv("../sensores_airenuevoleon.csv")
registros = registros.merge(sensores[["Sensor_id", "Latitud", "Longitud"]], on="Sensor_id", how="left")
registros = registros.dropna(subset=["Latitud", "Longitud"])

def gaussian_plume(lat, lon, velocidad, direccion, contaminante_val, n_puntos=50):
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    dir_rad = np.radians(direccion)
    distancias = np.linspace(0.001, 0.02, n_puntos)
    puntos = []
    for d in distancias:
        sigma = 0.002 + d * 2
        for _ in range(10):
            lat_disp = lat + d * np.cos(dir_rad)
            lon_disp = lon + d * np.sin(dir_rad)
            lat_disp += np.random.normal(0, sigma)
            lon_disp += np.random.normal(0, sigma)
            intensidad = contaminante_val * np.exp(-d * 100)
            puntos.append([lat_disp, lon_disp, intensidad / 150])
    return puntos

def mostrar_mapa_contaminante(fecha_objetivo: str, contaminante: str):
    if contaminante not in registros.columns:
        raise ValueError(f"Contaminante '{contaminante}' no encontrado.")

    datos_dia = registros[
        (registros["Fecha"].dt.strftime("%Y-%m-%d") == fecha_objetivo) &
        (registros[contaminante].notna()) &
        (registros[contaminante] >= 0)
    ].copy()

    print(f"Mostrando mapa para {contaminante} el {fecha_objetivo} con {len(datos_dia)} registros.")
    
    heat_data = []
    for _, fila in datos_dia.iterrows():
        velocidad = fila["VIENTOVEL"] if pd.notna(fila["VIENTOVEL"]) else 1
        direccion = fila["RS"] if pd.notna(fila["RS"]) else 0

        puntos = gaussian_plume(
            lat=fila["Latitud"],
            lon=fila["Longitud"],
            velocidad=velocidad,
            direccion=direccion,
            contaminante_val=fila[contaminante]
        )
        heat_data.extend(puntos)

    mapa = folium.Map(location=[25.67, -100.31], max_zoom=11, zoom_start=10, tiles="CartoDB positron")
    HeatMap(
        heat_data,
        radius=30,
        max_opacity=0.7,
        blur=20,
        gradient={0.1: 'blue', 0.4: 'lime', 0.6: 'yellow', 0.8: 'orange', 1.0: 'red'},
        min_opacity=0.3
    ).add_to(mapa)
    
    return mapa


In [7]:
'''
Fecha min = "2022-01-01"
Fecha max = "2025-04-04"

PM10
PM25
O3
'''
mapa = mostrar_mapa_contaminante("2025-03-22", "PM10")
mapa

Mostrando mapa para PM10 el 2025-03-22 con 11 registros.
